![Colab](https://www.tensorflow.org/images/colab_logo_32px.png) [Run in Google Colab](https://colab.research.google.com/drive/19cEeW5Ub3ii3nv_aJ7SzGllUQUMznylb?usp=sharing) ![GitHub](https://www.tensorflow.org/images/GitHub-Mark-32px.png) [View source on GitHub](https://github.com/TanyaYu/Curriculum-Project/blob/master/module4/TensorFlow_Lite.ipynb)

# TensorFlow Lite

TensorFlow Lite is a set of tools to help developers run TensorFlow models on *mobile*, *embedded*, and *IoT* devices. It enables on-device machine learning inference with low latency and a small binary size.

TensorFlow Lite is designed to make it easy to perform machine learning on devices, "at the edge" of the network, instead of sending data back and forth from a server. For developers, performing machine learning on-device can help improve:

- *Latency*: there's no round-trip to a server
- *Privacy*: no data needs to leave the device
- *Connectivity*: an Internet connection isn't required
- *Power consumption*: network connections are power hungry

TensorFlow Lite works with a huge range of devices, from tiny microcontrollers to powerful mobile phones.

## Development workflow
The workflow for using TensorFlow Lite involves the following steps:

1. **Pick a model**

  Bring your own TensorFlow model, find a model online, or pick a model from [Pre-trained models](https://www.tensorflow.org/lite/models) to drop in or retrain.

2. **Convert the model**

  If you're using a custom model, use the [TensorFlow Lite converter](https://www.tensorflow.org/lite/convert/index) and a few lines of Python to convert it to the TensorFlow Lite format.

3. **Deploy to your device**

  Run your model on-device with the [TensorFlow Lite interpreter](https://www.tensorflow.org/lite/guide/inference), with APIs in many languages.

4. **Optimize your model**

  Use our [Model Optimization Toolkit](https://www.tensorflow.org/lite/performance/model_optimization) to reduce your model's size and increase its efficiency with minimal impact on accuracy.

To learn more about using TensorFlow Lite in your project, see [Get started](https://www.tensorflow.org/lite/guide/get_started).

# Vehicles Classification Model
In this section we are going to build and train neural netowrk to classify vehicles. Then, we will convert the model into TensorFlow Lite format to be able to deploy it in a mobile app.

In [42]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dropout, Dense, Activation
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.optimizers import Adam, RMSprop, Adagrad, Adamax, SGD
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
import matplotlib.pyplot as plt
from tensorflow.keras.utils import get_file
from os.path import join, dirname, basename
from sklearn.model_selection import train_test_split
import pandas as pd
import glob

## Data Preparation

In [43]:
url = 'https://storage.googleapis.com/vehicle-dataset/vehicles_full.zip'
path_to_zip =get_file('vehicles_full.zip', origin=url, extract=True)
path = join(dirname(path_to_zip), 'vehicles_full')

In [44]:
files = glob.glob(path + '/*/*', recursive=True)
  
X = files
y = [basename(dirname(f)) for f in files]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

train_df = pd.DataFrame({"filename": X_train, "class": y_train}) 
test_df = pd.DataFrame({"filename": X_test, "class": y_test}) 

In [45]:
IMG_HEIGHT = 32 
IMG_WIDTH = 64
batch_size = 128
class_names = ['Class1', 'Class2', 'Class3', 'Class4', 'Class5', 'Class6']

In [46]:
image_generator = ImageDataGenerator(rescale=1./255,
                                     width_shift_range=.15,
                                     height_shift_range=.15,
                                     brightness_range=(0.1,0.9),
                                     zoom_range=0.3,
                                     channel_shift_range=150,
                                     horizontal_flip=True,
                                     validation_split=0.2) 

train_data_gen = image_generator.flow_from_dataframe(dataframe=train_df,
                                                     x_col="filename",
                                                     y_col="class",
                                                     subset="training",
                                                     shuffle=True,
                                                     seed=42,
                                                     batch_size=batch_size,
                                                     classes=class_names,
                                                     target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                     class_mode="categorical")
valid_data_gen = image_generator.flow_from_dataframe(dataframe=train_df,
                                                     x_col="filename",
                                                     y_col="class",
                                                     subset="validation",
                                                     shuffle=True,
                                                     seed=42,
                                                     batch_size=batch_size,
                                                     classes=class_names,
                                                     target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                     class_mode="categorical")

test_image_generator = ImageDataGenerator(rescale=1./255) 
test_data_gen = test_image_generator.flow_from_dataframe(dataframe=test_df,
                                                         x_col="filename",
                                                         y_col="class",
                                                         shuffle=False,
                                                         classes=class_names,
                                                         target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                         class_mode="categorical")

Found 9437 validated image filenames belonging to 6 classes.
Found 2359 validated image filenames belonging to 6 classes.
Found 2950 validated image filenames belonging to 6 classes.


/usr/local/lib/python3.6/dist-packages/keras_preprocessing/image/dataframe_iterator.py:282: UserWarning: Found 1 invalid image filename(s) in x_col="filename". These filename(s) will be ignored.
  .format(n_invalid, x_col)


## Build Model

In [47]:
model_top = ResNet50(
    include_top=False,
    weights=None,
    input_shape=(IMG_HEIGHT, IMG_WIDTH, 3))
x = GlobalAveragePooling2D()(model_top.output)  
x = Dropout(0.2)(x)
x = Dense(128)(x)
x = Dense(6)(x)
x = Activation('softmax')(x)
model = Model(model_top.input, x, name='resnet50')

## Train

In [48]:
early_stop = EarlyStopping(monitor='val_loss', patience=10, verbose=1)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', patience=5, verbose=1)

In [49]:
model.compile(optimizer=RMSprop(lr=0.0001),
              loss=CategoricalCrossentropy(),
              metrics=['accuracy'])

In [50]:
history = model.fit_generator(
    train_data_gen,
    validation_data=valid_data_gen,
    epochs=100,
    callbacks=[early_stop, reduce_lr]
)

Epoch 1/100
74/74 [==============================] - 16s 216ms/step - loss: 1.9418 - accuracy: 0.5501 - val_loss: 1.3126 - val_accuracy: 0.6838 - lr: 1.0000e-04
Epoch 2/100
74/74 [==============================] - 15s 203ms/step - loss: 1.6592 - accuracy: 0.5862 - val_loss: 1.1550 - val_accuracy: 0.6838 - lr: 1.0000e-04
Epoch 3/100
74/74 [==============================] - 15s 204ms/step - loss: 1.5784 - accuracy: 0.6186 - val_loss: 1.0612 - val_accuracy: 0.6838 - lr: 1.0000e-04
Epoch 4/100
74/74 [==============================] - 15s 204ms/step - loss: 1.3169 - accuracy: 0.6355 - val_loss: 1.2701 - val_accuracy: 0.6838 - lr: 1.0000e-04
Epoch 5/100
74/74 [==============================] - 15s 204ms/step - loss: 1.2846 - accuracy: 0.6488 - val_loss: 1.1551 - val_accuracy: 0.6838 - lr: 1.0000e-04
Epoch 6/100
74/74 [==============================] - 15s 205ms/step - loss: 1.1990 - accuracy: 0.6590 - val_loss: 1.1626 - val_accuracy: 0.6829 - lr: 1.0000e-04
Epoch 7/100
74/74 [===============

## Evaluate

In [59]:
test_eval = model.evaluate_generator(test_data_gen)
print('Test Accuracy      : %1.2f%%     Test loss      : %1.6f' % (test_eval[1]*100, test_eval[0]))

Test Accuracy      : 77.02%     Test loss      : 0.618203


#Save the Model in TFLite Format

In [53]:
import tensorflow.lite as lite
from tensorflow import TensorSpec, function

In [54]:
!mkdir "tflite_models"

mkdir: cannot create directory ‘tflite_models’: File exists


In [55]:
# Get the concrete function from the Keras model.
run_model = function(lambda x : model(x))

# Save the concrete function.
concrete_func = run_model.get_concrete_function(
    TensorSpec(model.inputs[0].shape, model.inputs[0].dtype)
)

## Standard TensorFlow Lite Model

In [56]:
TFLITE_MODEL = "tflite_models/vehicles.tflite"

# Convert the model to standard TensorFlow Lite model
converter = lite.TFLiteConverter.from_concrete_functions([concrete_func])
converted_tflite_model = converter.convert()
open(TFLITE_MODEL, "wb").write(converted_tflite_model)

94997908

## Quantized Model
Post-training quantization is a conversion technique that can reduce model size while also improving CPU and hardware accelerator latency, with little degradation in model accuracy. You can quantize an already-trained float TensorFlow model when you convert it to TensorFlow Lite format using the TensorFlow Lite Converter.

In [57]:
TFLITE_QUANT_MODEL = "tflite_models/vehicles_quant.tflite"

# Convert the model to quantized version with post-training quantization
converter = lite.TFLiteConverter.from_concrete_functions([concrete_func])
converter.optimizations = [lite.Optimize.OPTIMIZE_FOR_SIZE]
tflite_quant_model = converter.convert()
open(TFLITE_QUANT_MODEL, "wb").write(tflite_quant_model)

23848776

## Convert from Saved Model or from Keras Model

In [ ]:
# Converting a SavedModel to a TensorFlow Lite model.
converter = lite.TFLiteConverter.from_saved_model(saved_model_dir)
tflite_model = converter.convert()

# Converting a tf.Keras model to a TensorFlow Lite model.
converter = lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()